In [1]:
import ROOT as rt

Welcome to JupyROOT 6.28/00


In [2]:
load_path = "workspace/workspace_ggh_All_Zfit_no_e_cut_UL_calib_cat20.root"

In [3]:
file = rt.TFile.Open(load_path)

In [4]:
canvas = rt.TCanvas()
canvas.cd()
mass =  rt.RooRealVar("mh_ggh","mass (GeV)",100,81,101)
mass.setBins(50) 
frame = mass.frame()

In [5]:

bwWidth = rt.RooRealVar("bwz_Width" , "widthZ", 2.5, 0, 30)
bwmZ = rt.RooRealVar("bwz_mZ" , "mZ", 91.2, 90, 92)
sigma = rt.RooRealVar("sigma" , "sigma", 2, 0.0, 5.0)

bwWidth.setConstant(True)
# bwmZ.setConstant(True)
# sigma.setConstant(True)
#bwmZ.setConstant(True)

model1 = rt.RooVoigtian("Voigtian" , "Voigtian",mass, bwmZ, bwWidth, sigma)

exp_coeff = rt.RooRealVar("exp_coeff", "exp_coeff", 0.01, 0.00000001, 1) # positve coeff to get the peak shape we want 
# exp_coeff = rt.RooRealVar("exp_coeff", "exp_coeff", -0.01, -1, -0.00000001)
shift = rt.RooRealVar("shift", "Offset", 85, 75, 105)
shifted_mass = rt.RooFormulaVar("shifted_mass", "@0-@1", rt.RooArgList(mass, shift))
model2_1 = rt.RooExponential("Exponential", "Exponential", shifted_mass,exp_coeff)

center = rt.RooRealVar("center" , "center", 91.2, 75, 105)
erfc_coeff = rt.RooRealVar("erfc_coeff" , "erfc_coeff", 0.01, 0, 1.5)
erfc_in = rt.RooFormulaVar("erfc_in", "(@0 - @2) * @1", rt.RooArgList(mass, erfc_coeff, center)) 
model2_2 = rt.RooFit.bindPdf("erfc", rt.TMath.Erfc, erfc_in)


model2 = rt.RooProdPdf("erfc_exp", "erfc_exp", rt.RooArgList(model2_1, model2_2))
sigfrac = rt.RooRealVar("sigfrac", "sigfrac", 0.9, 0, 1.0)
model3 = rt.RooAddPdf("model3", "model3", rt.RooArgList(model1, model2),sigfrac)

bkgfrac = rt.RooFormulaVar("bkgfrac", "(1 - @0)", rt.RooArgList(sigfrac)) 
dummy_bkg_var = rt.RooProdPdf("dummy_bkg_var", "dummy_bkg_var", rt.RooArgList(bkgfrac, model2))  # pdf model just for plotting bkg on the plot with bkgfrac


[#0] WARNING:InputArguments -- RooProdPdf::addPdfs(dummy_bkg_var) list arg bkgfrac is not a PDF, ignored


In [ ]:
import time

start = time.time()
roo_dataset = file.w.data("data_Zfit_no_e_cut_UL_calib_cat20")
rt.EnableImplicitMT()
# model2_1.fitTo(roo_dataset, rt.RooFit.BatchMode("cpu"), Minos=True)
model3.fitTo(roo_dataset, rt.RooFit.BatchMode("cpu"))
# model2.fitTo(roo_dataset, rt.RooFit.BatchMode("cpu"))


end = time.time()
print(f"fitting elapsed time: {end - start}")

In [ ]:
# file.w.pdf("Voigtian_ggh_All").plotOn(frame)

file.w.data("data_Zfit_no_e_cut_UL_calib_cat20").plotOn(frame)
model3.plotOn(frame, rt.RooFit.LineColor(rt.kGreen))
model1.plotOn(frame, rt.RooFit.LineColor(rt.kRed))
model2.plotOn(frame, rt.RooFit.LineColor(rt.kBlack))
dummy_bkg_var.plotOn(frame, rt.RooFit.LineColor(rt.kBlue))
# model2_2.plotOn(frame, rt.RooFit.LineColor(rt.kRed))
frame.Draw()
canvas.Update()
canvas.Draw()
